In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
T = 3
Delta = 1000
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 100
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[12.81317951 11.09709382]


In [6]:
cp_truth

array([1000, 2000])

In [7]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [8]:
X_train_list.shape

(100, 3000, 100)

In [9]:
np.random.seed(0)

grid_n = 100
gamma_list = [80, 100]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_d = np.zeros(B)
loc_error_d = np.zeros(B)
K_d = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    dp_fit = dplr_cv_bt(grid_n, lam_list, gamma_list)
    cp_best, param_best, cp_best_cand = dp_fit.fit((Y_train, X_train), (Y_test, X_test))    
    run_time_d[b] = time.time() - start_time
    loc_error_d[b] = cp_distance(cp_best, cp_truth)
    K_d[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_d.mean(), run_time_d.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_d.std(), run_time_d.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_d < T - 1), sum(K_d == T - 1), sum(K_d > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 13.39, avg time: 167.41717215538026
std loc error: 14.408952078482322, std time: 3.326477034997443
K < K*: 0, K = K*: 100, K > K*: 0


In [10]:
loc_error_d

array([ 6.,  1., 33., 24.,  2., 15., 16., 13.,  4., 19.,  4., 23.,  4.,
        9., 11.,  6., 30., 10.,  0., 30., 17., 22.,  5.,  6., 67.,  2.,
        5., 15., 11., 17., 41., 20.,  3.,  5., 43.,  5., 52.,  5.,  4.,
       11., 59., 10.,  0., 10.,  5., 13.,  7., 10.,  8.,  0.,  8.,  2.,
        3., 30., 21.,  6., 25.,  7.,  3., 22., 12., 13.,  6., 40., 16.,
        0.,  4.,  2.,  1., 11.,  5.,  4.,  9., 12.,  4.,  7.,  7.,  8.,
        2., 18., 11., 19., 16.,  5.,  4.,  5., 18.,  6.,  9., 83., 38.,
        2.,  1.,  4.,  9.,  9., 10., 12., 22., 10.])

In [11]:
cp_best_list

[[1006, 2001],
 [1000, 2001],
 [1008, 2033],
 [1006, 2024],
 [1000, 1998],
 [1013, 1985],
 [996, 1984],
 [1000, 1987],
 [999, 1996],
 [1000, 2019],
 [997, 1996],
 [1000, 2023],
 [1004, 1997],
 [991, 1993],
 [1011, 2001],
 [1006, 1999],
 [1030, 2017],
 [990, 2003],
 [1000, 2000],
 [1002, 2030],
 [1017, 1989],
 [1017, 1978],
 [995, 2002],
 [994, 2006],
 [1000, 1933],
 [1000, 2002],
 [1005, 2004],
 [985, 1997],
 [1001, 1989],
 [996, 2017],
 [1041, 1997],
 [1002, 2020],
 [1000, 1997],
 [1005, 2000],
 [985, 1957],
 [995, 2002],
 [948, 2004],
 [1000, 1995],
 [996, 2002],
 [1002, 1989],
 [1059, 1998],
 [1004, 1990],
 [1000, 2000],
 [1010, 2000],
 [995, 1996],
 [998, 2013],
 [1007, 2004],
 [1010, 2000],
 [1008, 2000],
 [1000, 2000],
 [1008, 1997],
 [1000, 2002],
 [1000, 2003],
 [994, 1970],
 [979, 1987],
 [1000, 2006],
 [991, 2025],
 [1001, 2007],
 [997, 2001],
 [1022, 2007],
 [1009, 2012],
 [1011, 2013],
 [1000, 1994],
 [960, 1997],
 [1008, 1984],
 [1000, 2000],
 [1004, 2000],
 [998, 1998],
 

In [12]:
import pickle
with open(path + 'dplr_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_d, run_time_d, K_d], f)